In [ ]:
from ipymlwidgets.widgets import Canvas
from ipymlwidgets.utils import demo_image
import numpy as np

image = demo_image()
canvas = Canvas(width=image.shape[1], height=image.shape[0])
canvas.observe(print, "client_size")
canvas.css_width = "10%"
canvas.css_height = "auto"

canvas.set_image(image)
patch = np.full((10, 10, 4), [255, 0, 0, 255], dtype=np.uint8)
canvas.set_patch(25, 5, 10, 10, patch)
display(canvas)

import random
import ipywidgets as W
import time

def update_and_draw_random_patch(b):   

    #with canvas.hold():
    for i in range(2):
        
        # Draw a random colored patch at random position
        patch_size = random.randint(5, 15)
        x = random.randint(0, canvas.width - patch_size)
        y = random.randint(0, canvas.height - patch_size)
        
        # Random color
        color = [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255), 255]
        patch = np.full((patch_size, patch_size, 4), color, dtype=np.uint8)
        
        canvas.set_patch(x, y, patch_size, patch_size, patch)
        print(f"Drew {patch_size}x{patch_size} patch at ({x}, {y}) with color {color[:3]}")
        
# Create the button
update_button = W.Button(description="Update & Draw Random Patch")
update_button.on_click(update_and_draw_random_patch)

display(update_button)

canvas.observe(print, "mouse_enter")
canvas.observe(print, "mouse_leave")
canvas.observe(print, "mouse_click")
canvas.observe(print, "mouse_drag")
canvas.observe(print, "mouse_down")
canvas.observe(print, "mouse_up")


In [ ]:
from ipymlwidgets.widgets import Canvas
from ipymlwidgets.utils import demo_image
import numpy as np
import ipywidgets as W


image = demo_image(width=100, height=200)
canvas = Canvas(size=(image.shape[1], image.shape[0]), layers=2)
image[...,-1] = 100
image = np.random.randint(0, 255, image.shape, dtype=np.uint8)
image[...,-1] = 100

canvas.set_image(np.ones_like(image) * np.array([255, 0, 0, 255], dtype=np.uint8), layer=0)
canvas.set_image(image, layer=1)

canvas.stroke_color = "black"
canvas.stroke_width = 2
canvas.draw_rect(np.array([[10, 10, 20, 20]]))

t = W.Text(value="...", layout=W.Layout(width="100%"))
def set_text(x):    
    t.value = str(x["new"])
canvas.observe(set_text, names="mouse_click")

import ipywidgets as W
hbox = W.HBox(
    children=[canvas],
    layout=W.Layout(
        height='400px',  # This constrains the height
    )
)

display(hbox)
display(t)


In [ ]:
from ipymlwidgets.widgets import Image
from ipymlwidgets.utils import demo_image
import ipywidgets as W

import numpy as np

image = demo_image()
widget = Image(image)

hbox = W.HBox(
    children=[widget],
    layout=W.Layout(
        height='100px',  # This constrains the height
    )
)

display(hbox)

In [26]:
import ipywidgets as W
import anywidget
import traitlets

from ipymlwidgets.traits import SupportedTensor
from ipymlwidgets.utils import demo_image
from ipymlwidgets.widgets.ocr import ItemOCR


class DisplayList(anywidget.AnyWidget):
    """A container widget to display a list of ItemOCR widgets, with reorderable items."""

    children = traitlets.List(trait=traitlets.Instance(W.DOMWidget)).tag(sync=True, **W.widget_serialization)
    children_order = traitlets.List(trait=traitlets.Int()).tag(sync=True)
    _syn = traitlets.Int(0).tag(sync=False) # set by the backend to say that the update should be scheduled

    _esm = """
    async function unpack_models(model_ids, manager) {
        return Promise.all(
            model_ids.map(id => manager.get_model(id.slice("IPY_MODEL_".length)))
        );
    }
    async function render({ model, el }) {

        async function update_children(model, el, _syn) {
            const widget_manager = model.widget_manager;
            console.log("update_children");
            let model_ids = model.get("children");
            let children_models = await unpack_models(model_ids, widget_manager);
            el.innerHTML = "";
            // Append the child views in order
            for (let child_model of children_models) {
                let child_view = await widget_manager.create_view(child_model);
                el.appendChild(child_view.el);
            }
            const _next_syn = model.get("_syn");
            if (_syn != _next_syn) {
                await update_children(model, el, _next_syn).catch((error) => console.error(error));
            }
        }
        update_children(model, el, 0)
        
        el.classList.add("display-list");

        // Listen for changes to the children list
        model.on("change:_syn", () => {
            update_children(model, el, model.get("_syn")).catch((error) => console.error(error));
        });

    }
    export default { render };
    """

    _css = """
    .display-list {
        display: flex;
        flex-direction: column;  /* Stack items vertically */
        gap: 10px;               /* Add space between items */
    }
    """

    def __init__(self, items=None):
        super().__init__()
        if items is None:
            items = []
        self.children = items  # Set the initial list of items
        self.children_order = list(range(len(items)))
        self._sync = 0

    def reorder_items(self, new_order):
        self.children_order = new_order
        self.children = [self.children[i] for i in new_order]
        self._sync = self._sync + 1 # trigger the update


# Create and display the OCRItem widget with image and text
items = []
for i in range(10):
    items.append(ItemOCR(demo_image(), f"{i}"))

dl = DisplayList(items)
display(dl)

# import time
# time.sleep(1)

#dl.reorder_items([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])



DisplayList(children=[ItemOCR(children=[MultiCanvas((32, 32), layers=1), Text(value='0', layout=Layout(border_…

In [ ]:
from ipymlwidgets.widgets import ImageOCR
from ipymlwidgets.utils import demo_image
import numpy as np

image = demo_image()
image_ocr = ImageOCR(image, boxes=np.array([[0,0, 20, 20]]), texts=["Hello"])
image_ocr.image_widget.css_width = "50%"

display(image_ocr)

In [ ]:
from ipymlwidgets.widgets import ImageAnnotated
from ipymlwidgets.utils import demo_image
from PIL import Image 
import numpy as np
import ipywidgets as W

image = np.array(Image.fromarray(demo_image()).resize((256,256)))
print(image.shape)
widget = ImageAnnotated(image, boxes=np.array([[0,0,8,8]]), texts=["Hello"])
widget.css_width = "20%"
display(widget)

message = W.HTML(value="")
def update_message(x):  
    message.value = f"{x['new']}"
widget.observe(update_message, "selection")
display(message)
